# Getting Street-level Images

In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
from shapely.ops import linemerge

## Getting Market, Chestnut, and Walnut Streets from OSM and generating 250 m interval sample points in Philadelphia

### Merging connected OSM road segments into continuous street lines


In [3]:

CITY = "Philadelphia, Pennsylvania, USA"
STREETS = [
    "Market Street",
    "Chestnut Street",
    "Walnut Street"
]
DIST = 250  # distance control
OUT_DIR = "street_samples_250m_test"
OUT_CSV = "street_points_250m_test.csv"
OUT_GEOJSON = "street_points_250m.geojson"

ox.settings.log_console = True
ox.settings.use_cache = True

os.makedirs(OUT_DIR, exist_ok=True)

print(f"Downloading road network for {CITY} ...")
G = ox.graph_from_place(CITY, network_type="drive")
edges = ox.graph_to_gdfs(G, nodes=False, edges=True)
edges = edges[~edges.geometry.is_empty].copy()

edges_proj = edges.to_crs(epsg=3857)

rows = []

for street in STREETS:
    print(f"\n {street}")
    mask = edges_proj["name"].astype(str).str.contains(street, case=False, na=False)
    subset = edges_proj[mask].copy()

    if subset.empty:
        print(f" no {street} found")
        continue


    try:
        merged_geom = subset.geometry.union_all()
    except AttributeError:
        merged_geom = subset.geometry.unary_union  


    merged_lines = linemerge(merged_geom)

    if merged_lines.geom_type == "LineString":
        line_list = [merged_lines]
    else:
        line_list = list(merged_lines.geoms)

    seg_global_id = 0
    for line in line_list:
        length = line.length
        if length == 0:
            continue

        print(f"Length of the {seg_global_id} merged lined:  {length:.1f} m")


        distances = np.arange(0, length, DIST)
        for i, d in enumerate(distances):
            p_proj = line.interpolate(d)

            gdf_tmp = gpd.GeoSeries([p_proj], crs=edges_proj.crs).to_crs(epsg=4326)
            p = gdf_tmp.iloc[0]

            rows.append({
                "city":      CITY,
                "street":    street,
                "segment_id": seg_global_id,  
                "point_id":   i,               
                "distance_m": float(d),
                "lon":        p.x,
                "lat":        p.y
            })

        seg_global_id += 1

if not rows:
    raise ValueError("no point generated")

pts_df = pd.DataFrame(rows)
gdf_pts = gpd.GeoDataFrame(
    pts_df,
    geometry=gpd.points_from_xy(pts_df["lon"], pts_df["lat"]),
    crs="EPSG:4326"
)

csv_path = os.path.join(OUT_DIR, OUT_CSV)
geojson_path = os.path.join(OUT_DIR, OUT_GEOJSON)

gdf_pts.to_csv(csv_path, index=False)
gdf_pts.to_file(geojson_path, driver="GeoJSON")


 Market Street
Length of the 0 merged lined:  3956.5 m
Length of the 1 merged lined:  3951.1 m
Length of the 2 merged lined:  5286.6 m
Length of the 3 merged lined:  110.4 m
Length of the 4 merged lined:  75.7 m
Length of the 5 merged lined:  1482.9 m
Length of the 6 merged lined:  1206.2 m
Length of the 7 merged lined:  2300.5 m
Length of the 8 merged lined:  558.0 m

 Chestnut Street
Length of the 0 merged lined:  11798.4 m

 Walnut Street
Length of the 0 merged lined:  6971.4 m
Length of the 1 merged lined:  223.1 m
Length of the 2 merged lined:  206.6 m
Length of the 3 merged lined:  4826.4 m


In [5]:
import folium
import geopandas as gpd
import pandas as pd

street_gdfs = []

for street in STREETS:
    mask = edges["name"].astype(str).str.contains(street, case=False, na=False)
    gdf_line = edges[mask].copy()
    if gdf_line.empty:
        print(f"no {street} found")
        continue
    

    gdf_line = gdf_line.to_crs(epsg=4326)
    gdf_line["street_plot_name"] = street  
    street_gdfs.append(gdf_line)

if not street_gdfs:
    raise ValueError("no target streets found")

lines_all = gpd.GeoDataFrame(
    pd.concat(street_gdfs, ignore_index=True),
    crs="EPSG:4326"
)


In [10]:

centroid = lines_all.unary_union.centroid
center_lat = centroid.y
center_lon = centroid.x

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=14,
    tiles="CartoDB positron"
)

color_map = {
    "Market Street":  "#006C9A",  
    "Chestnut Street": "#F4A259", 
    "Walnut Street":  "#9A348E"   
}


for street_name, gdf_line in lines_all.groupby("street_plot_name"):
    
    folium.GeoJson(
        gdf_line,
        name=street_name,
        style_function=lambda feature, s=street_name: {
            "color": color_map.get(s, "black"),
            "weight": 4,
            "opacity": 0.9
        },
        tooltip=folium.GeoJsonTooltip(
            fields=["street_plot_name"],
            aliases=["Street:"]
        )
    ).add_to(m)

folium.LayerControl().add_to(m)

m 


C:\Users\12345\AppData\Local\Temp\ipykernel_26464\2654898470.py:1: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = lines_all.unary_union.centroid


### Generate sample points

In [11]:
import folium


center_lat = gdf_pts["lat"].mean()
center_lon = gdf_pts["lon"].mean()

m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=14,
    tiles="CartoDB positron"   # light color
)

color_map = {
    "Market Street":  "#006C9A",  # 孔雀蓝
    "Chestnut Street": "#F4A259", # 藏红金
    "Walnut Street":  "#9A348E"   # 桑葚紫
}


# points
for _, row in gdf_pts.iterrows():
    folium.CircleMarker(
        location=[row["lat"], row["lon"]],
        radius=3,
        color=color_map.get(row["street"], "black"),
        fill=True,
        fill_opacity=0.8,
        popup=f"{row['street']} | seg {row['segment_id']} | d={row['distance_m']:.0f} m"
    ).add_to(m)

folium.LayerControl().add_to(m)

m


In [12]:
print(f"\n {len(gdf_pts)} points generated")
print("CSV   ->", csv_path)
print("GeoJSON ->", geojson_path)


 178 points generated
CSV   -> street_samples_250m_test\street_points_250m_test.csv
GeoJSON -> street_samples_250m_test\street_points_250m.geojson


## Retrieving street-level images from Google Maps


### Setting camera heading to capture street-level images perpendicular to the street


In [16]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np

base_dir = "street_samples_250m"  
in_csv  = os.path.join(base_dir, "street_points_250m.csv")
out_csv = os.path.join(base_dir, "street_points_250m_with_heading.csv")

df = pd.read_csv(in_csv)

gdf = gpd.GeoDataFrame(
    df,
    geometry=gpd.points_from_xy(df["lon"], df["lat"]),
    crs="EPSG:4326"
)

gdf_proj = gdf.to_crs(epsg=3857)

def add_headings(group):
    group = group.sort_values("distance_m").copy()
    xs = group.geometry.x.values
    ys = group.geometry.y.values

    if len(group) < 2:
        group["heading_road"] = 0.0
        group["heading_sidewalk_right"] = 90.0
        group["heading_sidewalk_left"] = 270.0
        return group

    dx = np.roll(xs, -1) - xs
    dy = np.roll(ys, -1) - ys

    dx[-1] = xs[-1] - xs[-2]
    dy[-1] = ys[-1] - ys[-2]

    heading_road = (np.degrees(np.arctan2(dx, dy)) + 360) % 360

    group["heading_road"] = heading_road
    group["heading_sidewalk_right"] = (heading_road + 90) % 360
    group["heading_sidewalk_left"] = (heading_road - 90) % 360

    return group

gdf_proj_h = gdf_proj.groupby(
    ["street", "segment_id"],
    group_keys=False
).apply(add_headings)


result = pd.DataFrame(gdf_proj_h.drop(columns="geometry"))

result.to_csv(out_csv, index=False)

print(out_csv)
print("number of points with heading:", len(result))


street_samples_250m\street_points_250m_with_heading.csv
number of points with heading: 178


C:\Users\12345\AppData\Local\Temp\ipykernel_26464\2330363626.py:56: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ).apply(add_headings)


In [42]:
result.head()

,city,street,segment_id,point_id,distance_m,lon,lat,heading_road,heading_sidewalk_right,heading_sidewalk_left
0,"Philadelphia, Pennsylvania, USA",Market Street,0,0,0.0,-75.247183,39.962715,98.526560,188.526560,8.526560
1,"Philadelphia, Pennsylvania, USA",Market Street,0,1,250.0,-75.244962,39.962460,99.178797,189.178797,9.178797
2,"Philadelphia, Pennsylvania, USA",Market Street,0,2,500.0,-75.242745,39.962185,99.181719,189.181719,9.181719
3,"Philadelphia, Pennsylvania, USA",Market Street,0,3,750.0,-75.240528,39.961910,99.133392,189.133392,9.133392
4,"Philadelphia, Pennsylvania, USA",Market Street,0,4,1000.0,-75.238311,39.961637,99.219779,189.219779,9.219779


### Use API to get images

In [43]:
import os
import time
import requests
import pandas as pd

# Paths
base_dir = "street_samples_250m"
in_csv  = os.path.join(base_dir, "street_points_250m_with_heading.csv")
out_csv = os.path.join(base_dir, "street_points_250m_with_images.csv")

images_subdir = "gsv_sidewalk_250m"            # subfolder name for images
images_dir = os.path.join(base_dir, images_subdir)

API_KEY = "Your key"                  # <---  Google API key 
USE_RIGHT_SIDEWALK = True                      # True = use right sidewalk heading, False = left
SLEEP_SEC = 0.05                               # delay between requests (seconds)

# 2) Read points with headings
df = pd.read_csv(in_csv)
print("Loaded points with headings:", len(df))

os.makedirs(images_dir, exist_ok=True)
base_url = "https://maps.googleapis.com/maps/api/streetview"

image_files = []
image_paths = []
status_codes = []

for idx, row in df.iterrows():
    lat = row["lat"]
    lon = row["lon"]

    # Choose which sidewalk direction to use
    heading = row["heading_sidewalk_right"] if USE_RIGHT_SIDEWALK else row["heading_sidewalk_left"]
    heading_int = int(round(heading))

    # Make a stable, unique filename per row
    fname = f"idx{idx:04d}_seg{row['segment_id']}_pt{row['point_id']}_h{heading_int}.jpg"
    fpath = os.path.join(images_dir, fname)

    params = {
        "size": "640x640",
        "location": f"{lat},{lon}",
        "heading": heading,
        "pitch": 0,
        "fov": 90,
        "key": API_KEY
    }

    r = requests.get(base_url, params=params)
    status_codes.append(r.status_code)

    if r.status_code == 200:
        with open(fpath, "wb") as f:
            f.write(r.content)
    else:
        print(f"[WARNING] Download failed at idx={idx}, status={r.status_code}, text={r.text[:100]}")
        # If failed, keep an empty path so the row is still aligned
        fpath = ""
        fname = ""

    # Store relative path so the whole folder is portable
    image_files.append(fname)
    image_paths.append(os.path.join(images_subdir, fname) if fname else "")

    time.sleep(SLEEP_SEC)

# 3) Attach image info and online GSV URL to the DataFrame
df["image_file"] = image_files
df["image_path"] = image_paths
df["gsv_status"] = status_codes

df["gsv_url_sidewalk"] = df.apply(
    lambda r: (
        f"https://www.google.com/maps/@?api=1&map_action=pano"
        f"&viewpoint={r['lat']},{r['lon']}"
        f"&heading={r['heading_sidewalk_right'] if USE_RIGHT_SIDEWALK else r['heading_sidewalk_left']}"
    ),
    axis=1
)

df.to_csv(out_csv, index=False)

print("\nAll done.")
print("Final dataset with image paths saved to:")
print("  ", out_csv)
print("Images saved under folder:")
print("  ", images_dir)


Loaded points with headings: 178

All done.
Final dataset with image paths saved to:
   street_samples_250m\street_points_250m_with_images.csv
Images saved under folder:
   street_samples_250m\gsv_sidewalk_250m
